In [1]:
import csv
import pandas as pd
import urllib
import urllib2
import datetime

In [2]:
df = pd.read_csv('../Data/companylist.csv')
stock_tickers = df.Symbol

In [3]:
# Save results for each stock in a folder

#for i in range(len(stock_tickers)):
#    urllib.urlretrieve('http://finance.google.com/finance/getprices?i=600&p=300d&f=d,o,h,l,c,v&df=cpct&q=' + stock_tickers[0], "../Data_From_Script/getprices_" + stock_tickers[i] + "_long.txt");

In [4]:
dataDict = {};
times = [];
closing = [];

with open('../Data/getprices_AMD_long.txt', 'rb') as csvfile:
    pricereader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for i, row in enumerate(pricereader):
        if i > 6:
            dataDict[i] = row;
            values = row[0].split(',');
            times.append(values[0]);
            closing.append(float(values[1]));

In [5]:
timesUTC = [];

currUnixTime = 0;
currTime = None;

for i in range(len(times)):
    if times[i][0] == 'a':
        #print i;
        currUnixTime = int(times[i][1:]);
        #print currUnixTime;
        currTime = datetime.datetime.utcfromtimestamp(currUnixTime)
        #print currTime
        timesUTC.append(currTime);
    else:
        #print currUnixTime
        timesUTC.append(datetime.datetime.utcfromtimestamp(currUnixTime + int(times[i])));

In [6]:
#print timesUTC

In [7]:
import numpy as np;

##  Calculate moving average for all points over n data points in a list,
##  then also removes n - 1 data points and returns that as a separate list

def calculateMovingAverageList(data, n):    
    currentAverage = 0;
    internalValues = [];
    movingAverage = [];

    for i in range(n):
        # Append all values to internalValues;
        # print i;
        internalValues.append(data[i]);
        
    currentAverage = np.mean(internalValues);
    movingAverage.append(currentAverage);
        
    for i in range(n, len(data)):
        # print i;
        del internalValues[0]; # Delete first index
        internalValues.append(data[i]); # Add data at i;
        currentAverage = np.mean(internalValues);
        movingAverage.append(currentAverage);
        
    return movingAverage, data[n:];

In [8]:
movingAverage50, shorterData50 = calculateMovingAverageList(closing, 50);
timeStamp50 = timesUTC[50:];

In [9]:
movingAverage20, shorterData20 = calculateMovingAverageList(closing, 20);
timeStamp20 = timesUTC[20:];

In [10]:
movingAverage10, shorterData10 = calculateMovingAverageList(closing, 10);
timeStamp10 = timesUTC[10:];

In [11]:
movingAverage78, shorterData78 = calculateMovingAverageList(closing, 78);
timeStamp78 = timesUTC[78:];

In [12]:
movingAverage5, shorterData5 = calculateMovingAverageList(closing, 5);
timeStamp5 = timesUTC[5:];

In [13]:
movingAverage39, shorterData39 = calculateMovingAverageList(closing, 39);
timeStamp39 = timesUTC[39:];

In [14]:
movingAverage20, shorterData20 = calculateMovingAverageList(closing, 20);
timeStamp20 = timesUTC[20:];

In [15]:
len(timeStamp10), len(movingAverage10), len(closing)

(1972, 1973, 1982)

In [16]:
import matplotlib;
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: MacOSX


In [17]:
print timeStamp10[0];
print timeStamp10[1];
print timeStamp10[50];

print movingAverage10[0];
print movingAverage10[1];
print movingAverage10[50];

2017-04-24 13:30:10
2017-04-24 13:30:11
2017-04-24 13:32:44
13.13302
13.12452
13.41232


In [18]:
def fixTimeData(timeList, n, data, movingAverage):
    for i in range(n - 1):
        timeList.append(i);
        movingAverage.insert(0, 0);
        
    for j in range(i, len(data) - 1):
        timeList.append(j)
        
    return movingAverage;

In [19]:
fixTime5 = [];
fixTime20 = [];
fixTime39 = [];
fixTime78 = [];

movingAverage5 = fixTimeData(fixTime5, 5, closing, movingAverage5);
movingAverage20 = fixTimeData(fixTime20, 20, closing, movingAverage20);
movingAverage39 = fixTimeData(fixTime39, 39, closing, movingAverage39);
movingAverage78 = fixTimeData(fixTime78, 78, closing, movingAverage78);

In [20]:
##  Calculate decision points

In [21]:
def calculateSlope(movingAverageList):
    dX = 1.0;
    dY = [];
    
    for i in range(len(movingAverageList) - 1):
        dY.append(movingAverageList[i + 1] - movingAverageList[i]);
        
    dY.append(movingAverageList[i + 1]);
    slope = [dy / dX for dy in dY];
    return slope;

In [22]:
slope78 = calculateSlope(movingAverage78);
slope39 = calculateSlope(movingAverage39);
slope20 = calculateSlope(movingAverage20);
slope5 = calculateSlope(movingAverage5);

In [23]:
len(slope78), len(slope39), len(slope20), len(slope5)

(1982, 1982, 1982, 1982)

In [24]:
print len(movingAverage78)
print len(closing)

1982
1982


In [25]:
buyPoints = [];

for i in range(len(slope78)):
    if (slope78[i] >= 0) and (slope39[i] > 0) and (slope20[i] > 0) and (slope5[i] > 0) and (closing[i] > movingAverage78[i]):
        buyPoints.append(i);

In [26]:
print buyPoints

[39, 40, 41, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 65, 66, 68, 69, 78, 108, 109, 110, 111, 112, 113, 120, 121, 122, 123, 124, 125, 126, 127, 128, 138, 139, 140, 141, 142, 143, 157, 158, 159, 227, 230, 238, 399, 441, 442, 443, 444, 445, 448, 449, 455, 456, 457, 458, 479, 480, 481, 482, 483, 484, 488, 495, 496, 497, 498, 499, 503, 504, 505, 506, 507, 511, 512, 513, 514, 515, 516, 517, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 559, 560, 561, 562, 563, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 592, 599, 600, 601, 602, 603, 604, 611, 613, 614, 615, 616, 617, 624, 626, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 676, 677, 678, 769, 784, 787, 891, 893, 895, 904, 905, 906, 919, 923, 925, 946, 947, 948, 949, 950, 952, 953, 956, 979, 980, 981, 984, 986, 987,

In [27]:
sellPoints = [];

for i in range(len(slope78)):
    if (slope78[i] < 0) and (slope39[i] < 0) and (slope20[i] < 0) and (slope5[i] < 0) and (closing[i] < movingAverage78[i]):
        sellPoints.append(i);

In [28]:
for x in sellPoints:
    print timesUTC[x]

2017-04-24 13:39:39
2017-04-24 13:39:40
2017-04-24 13:39:42
2017-04-24 13:39:43
2017-04-24 13:39:54
2017-04-24 13:39:56
2017-04-24 13:39:57
2017-04-24 13:39:58
2017-04-24 13:40:02
2017-04-24 13:40:05
2017-04-24 13:40:06
2017-04-24 13:40:07
2017-04-24 13:40:08
2017-04-24 13:40:09
2017-04-24 13:40:11
2017-04-24 13:40:13
2017-05-01 13:30:04
2017-05-01 13:30:05
2017-05-01 13:30:07
2017-05-01 13:30:08
2017-05-01 13:30:09
2017-05-01 13:32:24
2017-05-01 13:32:25
2017-05-01 13:32:26
2017-05-01 13:32:27
2017-05-01 13:32:28
2017-05-01 13:32:34
2017-05-01 13:32:35
2017-05-01 13:32:36
2017-05-01 13:32:37
2017-05-01 13:32:38
2017-05-01 13:32:39
2017-05-01 13:32:43
2017-05-01 13:32:44
2017-05-01 13:32:46
2017-05-01 13:32:47
2017-05-01 13:32:48
2017-05-01 13:32:49
2017-05-01 13:32:50
2017-05-01 13:32:51
2017-05-01 13:32:52
2017-05-01 13:32:53
2017-05-01 13:32:54
2017-05-01 13:32:55
2017-05-01 13:32:56
2017-05-01 13:32:57
2017-05-01 13:32:58
2017-05-01 13:32:59
2017-05-01 13:33:00
2017-05-01 13:33:01


In [29]:
print buyPoints

[39, 40, 41, 43, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 63, 65, 66, 68, 69, 78, 108, 109, 110, 111, 112, 113, 120, 121, 122, 123, 124, 125, 126, 127, 128, 138, 139, 140, 141, 142, 143, 157, 158, 159, 227, 230, 238, 399, 441, 442, 443, 444, 445, 448, 449, 455, 456, 457, 458, 479, 480, 481, 482, 483, 484, 488, 495, 496, 497, 498, 499, 503, 504, 505, 506, 507, 511, 512, 513, 514, 515, 516, 517, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 559, 560, 561, 562, 563, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 592, 599, 600, 601, 602, 603, 604, 611, 613, 614, 615, 616, 617, 624, 626, 639, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672, 673, 674, 676, 677, 678, 769, 784, 787, 891, 893, 895, 904, 905, 906, 919, 923, 925, 946, 947, 948, 949, 950, 952, 953, 956, 979, 980, 981, 984, 986, 987,

In [30]:
print sellPoints

[163, 164, 166, 167, 178, 180, 181, 182, 186, 189, 190, 191, 192, 193, 195, 197, 204, 205, 207, 208, 209, 240, 241, 242, 243, 244, 250, 251, 252, 253, 254, 255, 259, 260, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 281, 282, 283, 284, 285, 311, 312, 313, 314, 315, 316, 320, 326, 328, 329, 330, 332, 334, 335, 336, 337, 338, 339, 344, 353, 362, 363, 364, 365, 366, 378, 380, 415, 417, 423, 430, 431, 432, 433, 435, 693, 708, 709, 710, 711, 712, 713, 717, 718, 719, 720, 721, 722, 734, 735, 736, 752, 806, 807, 808, 809, 810, 811, 813, 814, 815, 816, 817, 818, 819, 820, 821, 822, 823, 824, 825, 826, 841, 842, 843, 844, 845, 848, 857, 858, 859, 860, 868, 869, 911, 912, 933, 934, 960, 964, 1081, 1082, 1083, 1084, 1085, 1086, 1087, 1088, 1094, 1095, 1096, 1097, 1100, 1101, 1104, 1120, 1121, 1122, 1123, 1124, 1125, 1132, 1133, 1134, 1135, 1136, 1137, 1147, 1337, 1338, 1339, 1342, 1344, 1345, 1346, 1347, 1348, 1349, 1350, 1351, 1352, 1359, 1360, 1361, 

In [31]:
##  Find one buyPoint and one sellPoint

buy_sell_Points_tuple_list = []

for buyPoint in buyPoints:
    buy_sell_Points_tuple_list.append((buyPoint, "buy"));

for sellPoint in sellPoints:
    buy_sell_Points_tuple_list.append((sellPoint, "sell"));

In [32]:
buy_sell_Points_tuple_list.sort(key=lambda tup: tup[0])  # sorts in place

In [33]:
print buy_sell_Points_tuple_list

[(39, 'buy'), (40, 'buy'), (41, 'buy'), (43, 'buy'), (45, 'buy'), (46, 'buy'), (47, 'buy'), (48, 'buy'), (49, 'buy'), (50, 'buy'), (51, 'buy'), (52, 'buy'), (53, 'buy'), (54, 'buy'), (55, 'buy'), (56, 'buy'), (57, 'buy'), (58, 'buy'), (59, 'buy'), (63, 'buy'), (65, 'buy'), (66, 'buy'), (68, 'buy'), (69, 'buy'), (78, 'buy'), (108, 'buy'), (109, 'buy'), (110, 'buy'), (111, 'buy'), (112, 'buy'), (113, 'buy'), (120, 'buy'), (121, 'buy'), (122, 'buy'), (123, 'buy'), (124, 'buy'), (125, 'buy'), (126, 'buy'), (127, 'buy'), (128, 'buy'), (138, 'buy'), (139, 'buy'), (140, 'buy'), (141, 'buy'), (142, 'buy'), (143, 'buy'), (157, 'buy'), (158, 'buy'), (159, 'buy'), (163, 'sell'), (164, 'sell'), (166, 'sell'), (167, 'sell'), (178, 'sell'), (180, 'sell'), (181, 'sell'), (182, 'sell'), (186, 'sell'), (189, 'sell'), (190, 'sell'), (191, 'sell'), (192, 'sell'), (193, 'sell'), (195, 'sell'), (197, 'sell'), (204, 'sell'), (205, 'sell'), (207, 'sell'), (208, 'sell'), (209, 'sell'), (227, 'buy'), (230, 'bu

In [34]:
buy_list = [];
sell_list = [];

isBuy = True;

for value in buy_sell_Points_tuple_list:
    if isBuy == True and value[1] == 'buy':
        buy_list.append(value[0]);
        isBuy = False;
    elif isBuy == False and value[1] == 'sell':
        sell_list.append(value[0]);
        isBuy = True;

In [35]:
buy_list

[39, 227, 399, 441, 769, 891, 919, 946, 979, 1161, 1560]

In [36]:
sell_list

[163, 240, 415, 693, 806, 911, 933, 960, 1081, 1337, 1767]

In [37]:
##  Calculate earnings (from just sell minus buy)
earnings = [];
number_of_transactions = 0;

for i in range(len(sell_list)):
    earnings.append(closing[sell_list[i]] - closing[buy_list[i]]);
    number_of_transactions += 1;

#for i in range(1, len(buy_list)):
    # print buy_list[i], sell_list[i - 1];
    # print closing[buy_list[i]], closing
    #earnings.append(-(closing[buy_list[i]] - closing[sell_list[i - 1]]));
    #number_of_transactions += 1;

In [38]:
print earnings
print np.sum(earnings)
print np.mean(earnings)
print np.std(earnings)

[0.29999999999999893, -1.8699999999999992, -0.16499999999999915, 1.4024, -0.28419999999999845, -0.09600000000000009, -0.025000000000000355, -0.06990000000000052, 0.04969999999999963, 1.4097000000000008, 2.3900000000000006]
3.0417
0.276518181818
1.06758377301


In [39]:
## Price of transaction:
price_of_transactions = - 7 * number_of_transactions * 2;
print price_of_transactions;

-154


In [40]:
## Net sum with number_of_shares amount of stock
number_of_shares = 367;
print np.sum(earnings) * number_of_shares - price_of_transactions;

1270.3039


In [41]:
## Percentage gain
starting_amount = closing[buy_list[0]] * number_of_shares + 7;
final_amount = starting_amount + np.sum(earnings) * number_of_shares - price_of_transactions;

print final_amount;
print starting_amount;

print final_amount / starting_amount;

6096.0139
4825.71
1.26323668434


In [43]:
plt.plot(closing);
plt.plot(fixTime5, movingAverage5);
plt.plot(fixTime20, movingAverage20);
plt.plot(fixTime39, movingAverage39);
plt.plot(fixTime78, movingAverage78);

for buy in buy_list:
    plt.axvline(x=buy, color='g');
    
for sell in sell_list:
    plt.axvline(x=sell, color='r');

plt.xlim([0, 2500]);
plt.ylim([9.6, 15]);

#  Data from:
http://finance.google.com/finance/getprices?i=600&p=30d&f=d,o,h,l,c,v&df=cpct&q=AMD